Model a family tree so that when given a ‘name’ and
a ‘relationship’ as an input, the output are the people that
correspond to the relationship. 

Input: Person = Ish; Relation = Brothers
Expected Output: Chit, Vich


https://geektrust.sgp1.cdn.digitaloceanspaces.com/assets/images/family-tree.jpg

In [15]:
# has all the names of the person and to which family he/she belongs
# key- person name
# value- family's address
PERSON_ADDRESS = {}

MALE = 'M'
FEMALE = 'F'

In [16]:
# Util Functions
def is_main_member(func):
    def _is_main_member(person_name, relation, family):
        # some new statments or flow control
        if family.main_member.name != person_name:
            return "{0} does not have own {1}.".format(person_name, relation.lower())
        else:
            # calling the function
            return func(person_name, relation, family)
    return _is_main_member

@is_main_member
def get_father(person_name, relation, family):
    if family.parents.main_member.gender == MALE:
        return family.parents.main_member.name
    else:
        return family.parents.spouse.name

@is_main_member
def get_mother(person_name, relation, family):
    if family.parents.main_member.gender == FEMALE:
        return family.parents.main_member.name
    else:
        return family.parents.spouse.name

def get_children(person_name, relation, family):
    children = family.child_family
    children_names = [child.main_member.name for child in children]
    return children_names

def get_sons(person_name, relation, family):
    """Sons in the family"""
    children = family.child_family
    sons = [child.main_member.name for child in children if child.main_member.gender == MALE]
    return sons

def get_daughters(person_name, relation, family):
    """Daughters in the family."""
    return [child.main_member.name for child in family.child_family if child.main_member.gender == FEMALE]

@is_main_member
def get_brothers(person_name, relation, family):
    """Get the brother of the main member in the family."""
    sibling_families = family.parents.child_family
    brothers = []
    for sibling in sibling_families:
        # skip himself
        if sibling == family:
            continue
        if sibling.main_member.gender == MALE:
            brothers.append(sibling.main_member.name)
    return brothers

def get_brother_in_law(person_name, relation, family):
    # if he/she is not the main member, only then he/she has "in-laws".
    if family.main_member.name != person_name:
        sibling_families = family.parents.child_family
        brothers = []
        for sibling in sibling_families:
            # skip himself
            if sibling == family:
                continue
            if sibling.main_member.gender == MALE:
                brothers.append(sibling.main_member.name)
    else:
        return "{0} does not have own {1}.".format(person_name, relation.lower())
        
    return brothers

@is_main_member
def get_sisters(person_name, relation, family):
    sibling_families = family.parents.child_family
    brothers = []
    for sibling in sibling_families:
        # skip himself
        if sibling == family:
            continue
        if sibling.main_member.gender == FEMALE:
            brothers.append(sibling.main_member.name)
    return brothers

def get_sister_in_law(person_name, relation, family):
    # if he/she is not the main member, only then he/she has "in-laws".
    if family.main_member.name != person_name:
        sibling_families = family.parents.child_family
        sisters = []
        for sibling in sibling_families:
            # skip himself
            if sibling == family:
                continue
            if sibling.main_member.gender == FEMALE:
                sisters.append(sibling.main_member.name)
    else:
        return "{0} does not have own {1}.".format(person_name, relation.lower())
        
    return sisters

def get_grand_daughter(person_name, relation, family):
    g_children = []
    if not family.child_family:
        return g_children
    for child in family.child_family:
        if not child.child_family:
            continue
        for g_child in child.child_family:
            g_children.append(g_child.main_member.name)

    return g_children
@is_main_member
def get_cousins(person_name, relation, family):
    uncle_aunt_families = family.parents.parents.child_family
    cousin_name = []
    for uncle_aunt in uncle_aunt_families:
        for cousins in uncle_aunt.child_family:
            cousin_name.append(cousins.main_member.name)
    return cousin_name

def get_maternal_aunt(person_name, relation, family):
    uncle_aunt_families = family.parents.parents.child_family
    maternal_aunt = []
    for uncle_aunt in uncle_aunt_families:
        if uncle_aunt.main_member.gender == FEMALE:
            maternal_aunt.append(uncle_aunt.main_member.name)
    return maternal_aunt

In [17]:
# Relatioinship Mapper

RELATION = {
    "Paternal uncle": None,
    "Maternal uncle": None,
    "Paternal aunt":None,
    "Maternal aunt": get_maternal_aunt,
    "Sister-in law": get_sister_in_law,
    "Brother-in law": get_brother_in_law,
    "Cousins": get_cousins,
    "Father": get_father,
    "Mother": get_mother,
    "Children": get_children,
    "Sons": get_sons,
    "Daughters": get_daughters,
    "Brothers": get_brothers,
    "Sisters": get_sisters,
    "Grand daughter": get_grand_daughter
}

In [18]:
class Person(object):
    def __init__(self, name=None, gender=None):
        self.name = name
        self.gender = gender

In [19]:
class Family(object):
    def __init__(self, parents=None, main_member=None, spouse=None, child_family=None):
        self.parents = parents  # a Family object, 
        self.main_member = main_member  # a Person object
        self.spouse = spouse  # a Person object
        self.child_family = child_family or []  # list of child Family objects
        
        if self.main_member:
            self.update_address(self.main_member.name, self)
        if self.spouse:
            self.update_address(self.spouse.name, self)
    
    def update_address(self, name, address):
        PERSON_ADDRESS.update({name: address})
    
    def marriage(self, new_person):
        """Then main member of the family  is married to the new person."""
        self.spouse = new_person
        self.update_adddress(self.spouse.name, self) # the new person's family adddress is then updated
    
    def has_no_child_family(self):
        """Has no children or leaf node in a tree."""
        return not self.child_family
        

In [20]:
class Generation(object):
    def __init__(self, family=None):
        self.root_family = family
        # has all the names of the person and to which family he/she belongs
        self.person_dict = {}
        self.size = 0 # no. of families in this generations
    
    def get_family(self, name):
        try:
            return PERSON_ADDRESS[name]
        except:
            return None # the person does not have a family.
    
    def add_child_family(self, parent_name=None, child_family=None):
        """Add a child family in the Generation Tree.
        
        :param: parent_name- main member name of the parent in which to add the child family.
        :param: child_family- the familly object of the child."""
        # if the generation tree is empty
        if not self.root_family:
            self.root_family=child_family
            self.size += 1
            return
            
        if not parent_name and not child_family:
            return "Please specify the parent name and the child family"
        
        parent_family = self.get_family(parent_name)
        if not parent_family:
            return "No family found for the name: {0}".format(parent_name)
        
        
        child_family.parents = parent_family
        parent_family.child_family.append(child_family)
        self.size += 1

    def relationship(self, person_name=None, relation=None):
        family = self.get_family(person_name)
        return RELATION[relation](person_name, relation, family)


In [21]:
import collections
def display(tree):
    """Tree level order traversal"""
    if not tree:
        return
    nodes=collections.deque([tree])
    currentCount, nextCount = 1, 0
    while len(nodes)!=0:
        currentNode=nodes.popleft()
        currentCount-=1
        if currentNode.main_member:
            main_member = currentNode.main_member.name
            main_member_gender = currentNode.main_member.gender
        else:
            main_member = main_member_gender = ''

        if currentNode.spouse:
            spouse = currentNode.spouse.name
            spouse_gender = currentNode.spouse.gender
        else:
            spouse = spouse_gender = ''

        print("{0}({1})-{2}({3})".format(main_member, main_member_gender, spouse, spouse_gender), end=', ' )
        nodes = nodes + collections.deque(currentNode.child_family)
        nextCount += len( currentNode.child_family)
        
        if currentCount==0:
            #finished printing current level
            print('\n')
            currentCount, nextCount = nextCount, currentCount

In [22]:
import unittest

class TestFamily(unittest.TestCase):
    def setUp(self):
        self.family1 = Family()
        
        self.rob = Person(name='Rob', gender=MALE)
        self.rita = Person(name='Rita', gender=FEMALE)
        self.jeff = Person(name='Jeff', gender=MALE)

        self.sub_family = Family(main_member=self.jeff)
        self.family2 = Family(main_member=self.rob, spouse=self.rita, child_family=[self.sub_family])
        

    def test_has_no_child_family(self):
        self.assertEqual(self.family1.has_no_child_family(), True,
                         'Incorrect method implementation- has_no_child_family for family1')
        self.assertEqual(self.family2.has_no_child_family(), False,
                         'Incorrect method implementation- has_no_child_family for family2')
#     def test_add_child_family(self):
#         mary = Person(name='Mary', gender='F')
#         mary_family = Family(main_member=mary)
        
#         child_family_earlier = len(self.family1.child_family)
#         self.family1.add_child_family(mary_family)
        
#         self.assertEqual(child_family_earlier + 1, len(self.family1.child_family),
#                          'Incorrect method implementation- add_child_family() for family1')

        
# Run Tests
def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestFamily('test_has_no_child_family'))
#     suite.addTest(TestFamily('test_add_child_family'))
    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(suite())


.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [23]:
f = Family()

In [24]:
shan = Person(name='Shan', gender=MALE)
anga = Person(name='Anga', gender=FEMALE)

ish = Person(name='Ish', gender=MALE)
chit = Person(name='Chit', gender=MALE)
ambi = Person(name='Ambi', gender=FEMALE)
vich = Person(name='Vich', gender=MALE)
lika = Person(name='Lika', gender=FEMALE)
satya = Person(name='Satya', gender=FEMALE)
vyan = Person(name='Vyan', gender=MALE)

drita = Person(name='Drita', gender=MALE)
jaya = Person(name='Jaya', gender=FEMALE)
vrita = Person(name='Vrita', gender=MALE)

vila = Person(name='Vila', gender=MALE)
jnki = Person(name='Jnki', gender=FEMALE)
chika = Person(name='Chika', gender=FEMALE)
kpila = Person(name='Kpila', gender=MALE)

satvy = Person(name='Satvy', gender=FEMALE)
aswa = Person(name='Aswa', gender=MALE)
savya = Person(name='Savya', gender=MALE)
kripi = Person(name='Kripi', gender=FEMALE)
saayan = Person(name='Saayan', gender=MALE)
mina = Person(name='Mina', gender=FEMALE)

jta = Person(name='jta', gender=MALE)
driya = Person(name='Driya', gender=FEMALE)
mnu = Person(name='Mnu', gender=MALE)

lavnya = Person(name='Lavnya', gender=FEMALE)
gru = Person(name='Gru', gender=MALE)

kriya = Person(name='Kriya', gender=MALE)

misa = Person(name='Misa', gender=MALE)

In [25]:
shan_family = Family(main_member=shan, spouse=anga)

ish_family = Family(main_member=ish)
chit_family = Family(main_member=chit, spouse=ambi)
vich_family = Family(main_member=vich, spouse=lika)
satya_family = Family(main_member=satya, spouse=vyan)

drita_family = Family(main_member=drita, spouse=jaya)
vrita_family =Family(main_member=vrita)

vila_family = Family(main_member=vila, spouse=jnki)
chika_family = Family(main_member=chika, spouse=kpila)

satvy_family = Family(main_member=satvy, spouse=aswa)
savya_family = Family(main_member=savya, spouse=kripi)
saayan_family = Family(main_member=saayan, spouse=mina)

jta_family = Family(main_member=jta)
driya_family = Family(main_member=driya, spouse=mnu)

lavnya_family = Family(main_member=lavnya, spouse=gru)

kriya_family = Family(main_member=kriya)

misa_family = Family(main_member=misa)


In [26]:
g = Generation()
g.add_child_family(child_family=shan_family)
g.add_child_family(parent_name='Shan', child_family=ish_family)
g.add_child_family(parent_name='Shan', child_family=chit_family)
g.add_child_family(parent_name='Shan', child_family=vich_family)
g.add_child_family(parent_name='Shan', child_family=satya_family)

g.add_child_family(parent_name='Chit', child_family=drita_family)
g.add_child_family(parent_name='Chit', child_family=vrita_family)

g.add_child_family(parent_name='Vich', child_family=vila_family)
g.add_child_family(parent_name='Vich', child_family=chika_family)

g.add_child_family(parent_name='Satya', child_family=satvy_family)
g.add_child_family(parent_name='Satya', child_family=savya_family)
g.add_child_family(parent_name='Satya', child_family=saayan_family)

g.add_child_family(parent_name='Drita', child_family=jta_family)
g.add_child_family(parent_name='Drita', child_family=driya_family)

g.add_child_family(parent_name='Vila', child_family=lavnya_family)

g.add_child_family(parent_name='Savya', child_family=kriya_family)

g.add_child_family(parent_name='Saayan', child_family=misa_family)

In [27]:
display(g.root_family)

Shan(M)-Anga(F), 

Ish(M)-(), Chit(M)-Ambi(F), Vich(M)-Lika(F), Satya(F)-Vyan(M), 

Drita(M)-Jaya(F), Vrita(M)-(), Vila(M)-Jnki(F), Chika(F)-Kpila(M), Satvy(F)-Aswa(M), Savya(M)-Kripi(F), Saayan(M)-Mina(F), 

jta(M)-(), Driya(F)-Mnu(M), Lavnya(F)-Gru(M), Kriya(M)-(), Misa(M)-(), 



In [29]:
print(g.relationship(person_name='Ish', relation='Brothers'))
print(g.relationship(person_name='Chit', relation='Mother'))
print(g.relationship(person_name='Vich', relation='Father'))
print(g.relationship(person_name='Shan', relation='Sons'))
print(g.relationship(person_name='Shan', relation='Daughters'))
print(g.relationship(person_name='Ish', relation='Sisters'))
print(g.relationship(person_name='Shan', relation='Children'))
print(g.relationship(person_name='Ambi', relation='Brothers'))
print(g.relationship(person_name='Ambi', relation='Brother-in law'))
print(g.relationship(person_name='Jnki', relation='Sister-in law'))
print(g.relationship(person_name='Kriya', relation='Maternal aunt'))

['Chit', 'Vich']
Anga
Shan
['Ish', 'Chit', 'Vich']
['Satya']
['Satya']
['Ish', 'Chit', 'Vich', 'Satya']
Ambi does not have own brothers.
['Ish', 'Vich']
['Chika']
['Satvy']
